## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
import cv2
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing data from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
# Setting the project path
project_path = '/content/drive/My Drive/dog_breeds_data/'

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
    z.extractall()

Repeat the same step for test.zip

In [0]:
with ZipFile(project_path+'test.zip', 'r') as z:
    z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
    z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
    z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [8]:
labels_df = pd.read_csv('labels.csv')
labels_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [9]:
breed_count = labels_df['breed'].value_counts()
breed_count

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
brabancon_griffon        67
komondor                 67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64

In [10]:
num_classes = len(labels_df.breed.unique())
print(f'Number of dog breeds: {num_classes}')

Number of dog breeds: 120


### Get one-hot encodings of labels

In [0]:
le = preprocessing.LabelEncoder()
enc_labels = le.fit_transform(labels_df['breed'])

In [0]:
enc_labels = tf.keras.utils.to_categorical(enc_labels, num_classes=num_classes, dtype='int')

In [13]:
enc_labels.shape

(10222, 120)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [14]:
X_train = []
y_train = []

for file, label in tqdm(labels_df.values):
    y_train.append(label)
    image = cv2.imread(f'./train/{file}.jpg')
    image = cv2.resize(image, (128, 128))
    X_train.append(image)

100%|██████████| 10222/10222 [00:31<00:00, 329.28it/s]


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
X_train = np.array(X_train, np.float32)/255.0
x_train_data = np.reshape(X_train, (len(X_train), 128, 128, 3))

In [16]:
x_train_data.shape

(10222, 128, 128, 3)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [17]:
train_x, val_x, train_y, val_y = train_test_split(x_train_data, enc_labels, test_size=0.2, random_state=7)
print ("No. of images in train dataset: ", len(train_x))
print ("No. of images in Validation dataset: ", len(val_x))

No. of images in train dataset:  8177
No. of images in Validation dataset:  2045


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [18]:
test_img = pd.read_csv('/content/sample_submission.csv')['id']
test_img.shape

(10357,)

Run the below code to load the test image files in x_test_feature

In [19]:
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (128, 128)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [00:14<00:00, 693.35it/s]


In [20]:
x_test_feature[0]

array([[104,  98,  93, ...,  45,  99,  87],
       [ 83,  71, 105, ...,  75,  73,  93],
       [ 72, 105, 115, ...,  75,  54, 128],
       ...,
       [ 79,  40,  86, ...,  35,  36,  21],
       [ 39, 117, 200, ...,  25,  37,  50],
       [ 32,  46,  16, ...,   7,  49,  37]], dtype=uint8)

Normalize the test data and convert it into 4 dimensions

In [0]:
# x_test_data = np.array(x_test_feature).astype(float)/255.0
# x_test_data = np.reshape(x_test_data, (len(x_test_feature), 128, 128, 3))
x_test_data = tf.keras.applications.vgg16.preprocess_input(np.array(x_test_feature, np.float32))

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [22]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=64, input_shape = (128,128,3), kernel_size=(5,5), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten()) 

# fully connected layer
model.add(tf.keras.layers.Dense(256, activation = 'relu'))
model.add(tf.keras.layers.Dense(120, activation = 'softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 64)      4864      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 62, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 128)       512       
_________________________________________________________________
flatten (Flatten)            (None, 115200)            0

In [0]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [0]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1, mode='auto')
callback_list = [early_stopping]

### Use batch_size = 128 and epochs = 10 and execute the model

In [26]:
model.fit(train_x, train_y, epochs=10, validation_data=(val_x, val_y), verbose = 1, batch_size=128,callbacks=callback_list)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 68s 8ms/sample - loss: 12.1879 - acc: 0.0179 - val_loss: 11.1938 - val_acc: 0.0093
Epoch 2/10
8177/8177 [==============================] - 67s 8ms/sample - loss: 4.6775 - acc: 0.1526 - val_loss: 16.5534 - val_acc: 0.0142
Epoch 3/10
8177/8177 [==============================] - 67s 8ms/sample - loss: 3.0389 - acc: 0.3469 - val_loss: 29.0249 - val_acc: 0.0127
Epoch 4/10
8177/8177 [==============================] - 67s 8ms/sample - loss: 2.1283 - acc: 0.5261 - val_loss: 34.6517 - val_acc: 0.0112
Epoch 5/10
8177/8177 [==============================] - 67s 8ms/sample - loss: 1.4168 - acc: 0.6600 - val_loss: 32.0146 - val_acc: 0.0127
Epoch 00005: early stopping


In [27]:
train_res = model.evaluate(train_x, train_y, verbose=0)
print(f'Train Accuracy: {train_res[1]}')

val_res = model.evaluate(val_x, val_y, verbose=0)
print(f'Validation Accuracy: {val_res[1]}')

Train Accuracy: 0.033141739666461945
Validation Accuracy: 0.012713936157524586


#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,  
    samplewise_center=False, 
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=50,  
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    horizontal_flip=False,  
    vertical_flip=False)

In [0]:
train_datagen.fit(train_x)

In [0]:
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,  
    samplewise_center=False, 
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=50,  
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    horizontal_flip=False,  
    vertical_flip=False)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
train_generator = train_datagen.flow(train_x, train_y, batch_size=9)
val_generator = val_datagen.flow(val_x, val_y, batch_size=9)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [32]:
model.fit_generator(train_generator, epochs=10, steps_per_epoch=300, verbose=1,validation_data=val_generator, validation_steps = 300)

Epoch 1/10
300/300 [==============================] - 49s 162ms/step - loss: 7.5643 - acc: 0.0145 - val_loss: 4.7917 - val_acc: 0.0100
Epoch 2/10
300/300 [==============================] - 49s 162ms/step - loss: 4.8002 - acc: 0.0119 - val_loss: 4.7894 - val_acc: 0.0097
Epoch 3/10
300/300 [==============================] - 47s 157ms/step - loss: 4.7883 - acc: 0.0104 - val_loss: 4.8002 - val_acc: 0.0104
Epoch 4/10
300/300 [==============================] - 49s 162ms/step - loss: 4.7861 - acc: 0.0141 - val_loss: 23.5815 - val_acc: 0.0137
Epoch 5/10
300/300 [==============================] - 49s 163ms/step - loss: 4.7944 - acc: 0.0107 - val_loss: 4.7911 - val_acc: 0.0089
Epoch 6/10
300/300 [==============================] - 49s 162ms/step - loss: 4.7836 - acc: 0.0107 - val_loss: 4.7879 - val_acc: 0.0082
Epoch 7/10
300/300 [==============================] - 49s 162ms/step - loss: 4.7809 - acc: 0.0130 - val_loss: 4.7852 - val_acc: 0.0097
Epoch 8/10
300/300 [==============================] - 

In [33]:
train_res = model.evaluate(train_x, train_y, verbose=0)
print(f'Train Accuracy: {train_res[1]}')

val_res = model.evaluate(val_x, val_y, verbose=0)
print(f'Validation Accuracy: {val_res[1]}')

Train Accuracy: 0.009538950398564339
Validation Accuracy: 0.009290953166782856


# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
# from tf.keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= tf.keras.applications.vgg16.VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),include_top=False, pooling='avg')

Print the summary of the base_model

In [41]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

Referring to the URL
https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/

In [42]:
base_model.layers.pop()

In [0]:
spec = base_model.layers[-1].output_shape[1:]

In [44]:
base_model.layers[-1].output

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(?, 512) dtype=float32>

In [0]:
from tensorflow.keras import layers
model1 = base_model.layers[-1].output

model1 = tf.keras.layers.Flatten(input_shape=spec)(model1)
model1 = tf.keras.layers.Dense(1024, activation='relu')(model1)
model1 = tf.keras.layers.BatchNormalization()(model1)
model1 = tf.keras.layers.Dense(256, activation='relu')(model1)
model1 = tf.keras.layers.BatchNormalization()(model1)
model1 = tf.keras.layers.Dense(num_classes, activation='softmax')(model1)

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
for layer in base_model.layers:
    layer.trainable = False

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
inp = base_model.input

In [0]:
model2 = tf.keras.Model(inputs=inp, outputs=model1)

In [0]:
model2.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [54]:
model2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [56]:
model2.fit(train_x, train_y, epochs=10, validation_data=(val_x, val_y), batch_size=128, verbose = 1, callbacks=callback_list)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 101s 12ms/sample - loss: 4.2904 - acc: 0.0962 - val_loss: 4.4390 - val_acc: 0.0704
Epoch 2/10
8177/8177 [==============================] - 100s 12ms/sample - loss: 3.0282 - acc: 0.2863 - val_loss: 4.1675 - val_acc: 0.1232
Epoch 3/10
8177/8177 [==============================] - 100s 12ms/sample - loss: 2.5075 - acc: 0.3935 - val_loss: 3.8839 - val_acc: 0.1594
Epoch 4/10
8177/8177 [==============================] - 101s 12ms/sample - loss: 2.1237 - acc: 0.4921 - val_loss: 3.7510 - val_acc: 0.1609
Epoch 5/10
8177/8177 [==============================] - 100s 12ms/sample - loss: 1.7892 - acc: 0.5776 - val_loss: 3.6875 - val_acc: 0.1511
Epoch 6/10
8177/8177 [==============================] - 100s 12ms/sample - loss: 1.4941 - acc: 0.6579 - val_loss: 3.6556 - val_acc: 0.1570
Epoch 7/10
8177/8177 [==============================] - 100s 12ms/sample - loss: 1.2394 - acc: 0.7248 - val_loss: 3.61

In [57]:
model2.fit(train_x, train_y, epochs=50, validation_data=(val_x, val_y), batch_size=128, verbose = 1, callbacks=callback_list)

Train on 8177 samples, validate on 2045 samples
Epoch 1/50
8177/8177 [==============================] - 100s 12ms/sample - loss: 0.4254 - acc: 0.9424 - val_loss: 4.2763 - val_acc: 0.1809
Epoch 2/50
8177/8177 [==============================] - 102s 12ms/sample - loss: 0.3013 - acc: 0.9687 - val_loss: 4.3430 - val_acc: 0.1844
Epoch 3/50
8177/8177 [==============================] - 100s 12ms/sample - loss: 0.2069 - acc: 0.9884 - val_loss: 4.3926 - val_acc: 0.1790
Epoch 4/50
8177/8177 [==============================] - 100s 12ms/sample - loss: 0.1356 - acc: 0.9956 - val_loss: 4.3744 - val_acc: 0.1912
Epoch 5/50
8177/8177 [==============================] - 101s 12ms/sample - loss: 0.0897 - acc: 0.9983 - val_loss: 4.2864 - val_acc: 0.2064
Epoch 6/50
8177/8177 [==============================] - 101s 12ms/sample - loss: 0.0581 - acc: 0.9993 - val_loss: 4.2891 - val_acc: 0.2044
Epoch 7/50
8177/8177 [==============================] - 101s 12ms/sample - loss: 0.0422 - acc: 0.9993 - val_loss: 4.40

In [58]:
train_res = model2.evaluate(train_x, train_y, verbose=0)
print(f'Train Accuracy: {train_res[1]}')

val_res = model2.evaluate(val_x, val_y, verbose=0)
print(f'Validation Accuracy: {val_res[1]}')

Train Accuracy: 0.9996331334114075
Validation Accuracy: 0.20488998293876648
